# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [24]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import time
import json

# Import API key
from api_keys import geoapify_key

In [25]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
# Check the data types and display the first few rows
#print(city_data_df.dtypes)
#print(city_data_df.head())
# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,arraial do cabo,-22.9661,-42.0278,22.11,76,0,3.01,BR,1720074432
1,1,pangody,65.8500,74.4859,22.92,50,47,2.91,RU,1720074432
2,2,trecase,40.7694,14.4377,23.15,73,0,1.54,IT,1720074432
3,3,port-aux-francais,-49.3500,70.2167,1.01,60,18,4.16,TF,1720074433
4,4,den burg,53.0542,4.7972,15.54,67,45,9.26,NL,1720074433


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [26]:


map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500, 
    size = "Humidity",
    #scale = 0.01,
    color = "City",
    alpha = 0.6,
    legend='right',  # Place the legend on the left side
    cmap='Category20'  # Use a colormap suitable for categorical data
)


# Display the map plot
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [35]:
# Narrow down cities that fit criteria and drop any results with null values

# Define ideal weather conditions
ideal_temp_min = 20
ideal_temp_max = 30
max_humidity = 60
max_cloudiness = 60
max_wind_speed = 5
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] >= ideal_temp_min) & 
    (city_data_df['Max Temp'] <= ideal_temp_max) &
    (city_data_df['Humidity'] < max_humidity) &
    (city_data_df['Cloudiness'] < max_cloudiness) &
    (city_data_df['Wind Speed'] < max_wind_speed)
]
# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,pangody,65.8500,74.4859,22.92,50,47,2.91,RU,1720074432
57,57,howard springs,-12.4970,131.0470,29.43,34,39,3.80,AU,1720074492
77,77,sereflikochisar,38.9393,33.5386,25.83,33,0,3.16,TR,1720074495
103,103,abashiri,44.0213,144.2697,28.16,36,0,4.63,JP,1720074500
132,132,revda,56.8010,59.9303,27.68,57,11,3.27,RU,1720074554


### Step 3: Create a new DataFrame called `hotel_df`.

In [36]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
1,pangody,RU,65.8500,74.4859,50,
57,howard springs,AU,-12.4970,131.0470,34,
77,sereflikochisar,TR,38.9393,33.5386,33,
103,abashiri,JP,44.0213,144.2697,36,
132,revda,RU,56.8010,59.9303,57,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [37]:
# Set parameters to search for a hotel
radius = 10000
params = {
        "categories": "accommodation.hotel",
        #"filter": f"circle:{lng},{lat},{radius}",
        #"bias": f"proximity:{lng},{lat}",
        "limit": 1,
        "apiKey": geoapify_key
    }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
    # run a request using our params dictionary
    response = requests.get(base_url, params=params)

    # print the response url, avoid doing for public github repos in order to avoid exposing key
    #print(response.url)

    # Make and API request using the params dictionaty
    #name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    #Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        #If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
pangody - nearest hotel: Газпром энерго
howard springs - nearest hotel: Rydges Palmerston
sereflikochisar - nearest hotel: Has Hotel
abashiri - nearest hotel: ホテル オホーツク・イン
revda - nearest hotel: Металлург
santo antonio do leverger - nearest hotel: No hotel found
nizhneangarsk - nearest hotel: No hotel found
chiri-yurt - nearest hotel: No hotel found
ikovka - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found
mount isa - nearest hotel: Ibis Styles
alvorada - nearest hotel: No hotel found
bullas - nearest hotel: No hotel found
yangambi - nearest hotel: No hotel found
khovd - nearest hotel: Khovd hotel
severobaykal'sk - nearest hotel: Черембасс
gori - nearest hotel: ინტურისტი
martuk - nearest hotel: No hotel found
lefkada - nearest hotel: Ianos
suda - nearest hotel: No hotel found
susuman - nearest hotel: No hotel found
leamington - nearest hotel: Days Inn
sao vicente - nearest hotel: Pousada Vitória
pontal - nearest hotel: No hotel found
changji -

,City,Country,Lat,Lng,Humidity,Hotel Name
1,pangody,RU,65.8500,74.4859,50,Газпром энерго
57,howard springs,AU,-12.4970,131.0470,34,Rydges Palmerston
77,sereflikochisar,TR,38.9393,33.5386,33,Has Hotel
103,abashiri,JP,44.0213,144.2697,36,ホテル オホーツク・イン
132,revda,RU,56.8010,59.9303,57,Металлург
147,santo antonio do leverger,BR,-15.8656,-56.0767,42,No hotel found
153,nizhneangarsk,RU,55.7833,109.5500,52,No hotel found
157,chiri-yurt,RU,43.0867,45.7406,48,No hotel found
206,ikovka,RU,55.6089,64.9375,58,No hotel found
223,tsiombe,MG,-25.3000,45.4833,49,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [38]:

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500, 
    size = "Humidity",
    #scale = 0.01,
    color = "City",
    alpha = 0.6,
    legend='right',  # Place the legend on the left side
    #cmap='Category20'  # Use a colormap suitable for categorical data
    hover_cols=['City', 'Country', 'Hotel Name', 'Humidity']
)


# Display the map plot
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)